In [3]:
import pandas as pd
import numpy as np
import numba as nb
import networkx as nx
from tqdm.auto import tqdm, trange
import gc
import multiprocessing as mp
import random

In [4]:
graph_data = pd.read_csv('conceptnet.en.csv',sep='\t',header=None)
train_data = pd.read_json('train_rand_split.jsonl',lines=True)
dev_data = pd.read_json('dev_rand_split.jsonl',lines=True)
test_data = pd.read_json('test_rand_split_no_answers.jsonl',lines=True)

In [5]:
graph_data = graph_data.dropna()
graph_data[1] = graph_data[1].apply(lambda x: x.replace('_',' '))
graph_data[2] = graph_data[2].apply(lambda x: x.replace('_',' '))
graph_concepts = set(graph_data[1].tolist())

In [6]:
def parse_concepts(json_s):
    concepts = [json_s['question_concept']]
    concepts += [x['text'] for x in json_s['choices']] 
    return concepts

train_concepts = np.concatenate([parse_concepts(x) for x in train_data['question']]).tolist()
test_concepts = np.concatenate([parse_concepts(x) for x in test_data['question']]).tolist()
dev_concepts = np.concatenate([parse_concepts(x) for x in dev_data['question']]).tolist()
all_concepts = train_concepts + test_concepts + dev_concepts

In [7]:
len(all_concepts), len(graph_concepts)

(72612, 600641)

In [8]:
all_concepts = [x for x in all_concepts if x in graph_concepts]

In [9]:
all_concepts[0]

'punishing'

In [10]:
graph_data['relation_maper'] = graph_data.apply(lambda x: {'rel_type':x[0],'target_concept':x[2],'weights':x[3]},axis=1)

In [11]:
graph = graph_data.groupby(1)['relation_maper'].agg(list)

In [12]:
graph = graph.to_dict()

In [13]:
#del graph_data
#del graph_concepts
#gc.collect()

In [14]:
class RandomWalks():
    def __init__(self, graph, edges, n_walks=100, death=10,return_back=False):
        self.graph = graph
        self.edges = edges
        self.n_walks = n_walks
        self.death = death
        self.return_back = return_back 
        self.graph_edges = set(self.graph.keys())
    
    def get_walk(self,i):
        all_walks = []
        for edge in self.edges:
            walk = [edge]
            walk_s = [edge]
            c = 0

            while len(walk) < self.death:
                if walk[-1] not in self.graph_edges:
                        break
                        
                neighbours = self.graph[walk[-1]]
                weights = [x['weights'] for x in neighbours]
                neigh = random.choices(neighbours, weights=weights, k=1)[0]
                
                if not self.return_back and neigh['target_concept'] in walk:
                    c += 1
                    if c > 100:
                        break
                    continue
                    
                walk.append(neigh['target_concept'])
                walk_s += [neigh['rel_type'],neigh['target_concept']]
                
            all_walks.append(walk_s)
            
        return all_walks
    
    def process(self):
        pool = mp.Pool(mp.cpu_count())
        walks = pool.map(self.get_walk,trange(self.n_walks))
        return walks

In [15]:
walker = RandomWalks(
    graph=graph,
    edges=all_concepts,
    n_walks=5,
    death=10
)

In [16]:
walks = walker.process()

  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
walks_flatten = []
for w in walks:
    walks_flatten += w

In [18]:
len(walks_flatten)

302015

In [19]:
walks_flatten = [x for x in walks_flatten if len(x) > 1]

In [20]:
len(walks_flatten)

301990

In [23]:
import pickle
with open('random_walks.pkl','wb') as f:
    pickle.dump(walks_flatten,f)

In [19]:
import pickle
with open('random_walks.pkl','rb') as f:
    walks_flatten = pickle.load(f)

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
from src import WordBaseMLMDataCollator

In [24]:
collator = WordBaseMLMDataCollator(tokenizer)

In [32]:
shp = []
for i in trange(10_000):
    labels = collator(walks_flatten[i])['labels']
    ls = len(labels)
    labels = labels[labels != -100]
    shp.append([len(labels),ls])

  0%|          | 0/10000 [00:00<?, ?it/s]

In [20]:
np.random.random(10)

array([0.40892281, 0.33366749, 0.0188502 , 0.70509073, 0.1354656 ,
       0.02237957, 0.09535725, 0.29497756, 0.02969118, 0.33848867])

In [35]:
from math import factorial
n=0
(factorial(20) / (factorial(20-n) * factorial(n))) * (0.15)**n * (0.85)**(20-n)

0.03875953108451433

In [57]:
arr =np.random.random(10)

In [61]:
enc = collator(walks_flatten[100])

In [62]:
enc

{'input_ids': tensor([     1,   4433,    303,  62261,    818,   1144,    725,    671,    269,
            452,    728,    719,    269,    452, 128000, 128000,    269,    452,
           1727,   1144,    725,   2269,   3178,   1580,   1653,  16327,      2,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
     

In [65]:
tokenizer.decode(enc['input_ids'].tolist())

'[CLS] satisfaction hascontext law relatedto government isa social control isa[MASK][MASK] isa act relatedto map capableof guide traveler[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [66]:
tokenizer.decode([max(0,x) for x in enc['labels'].tolist()])

'[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD] group action[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'